In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Load Data

### Initialize Path Constants

In [2]:
RAW_DATA_PATH = '../data/raw'

### Load CSV Files

In [3]:
train_df = pd.read_csv('{}/tubes2_HeartDisease_train.csv'.format(RAW_DATA_PATH))
test_df = pd.read_csv('{}/tubes2_HeartDisease_test.csv'.format(RAW_DATA_PATH))

## Rename Column Names & Convert '?' to NaN

In [4]:
from copy import deepcopy

test_columns_replacement = {
    'Column1': 'age',
    'Column2': 'sex',
    'Column3': 'chest_pain_type',
    'Column4': 'resting_blood_pressure',
    'Column5': 'serum_cholestrol',
    'Column6': 'fasting_blood_sugar',
    'Column7': 'resting_ECG',
    'Column8': 'max_heart_rate_achieved',
    'Column9': 'excercise_induced_angina',
    'Column10': 'ST_depression',
    'Column11': 'peak_exercise_ST_segment',
    'Column12': 'num_of_major_vessels',
    'Column13': 'thal',
}

train_columns_replacement = test_columns_replacement.copy()
train_columns_replacement['Column14'] = 'heart_disease_diagnosis'

train_df = train_df.rename(columns=train_columns_replacement).replace('?',np.NaN)

test_df = test_df.rename(columns=test_columns_replacement).replace('?',np.NaN)

combine = [train_df,test_df]

In [5]:
train_df.head()

,age,sex,chest_pain_type,resting_blood_pressure,serum_cholestrol,fasting_blood_sugar,resting_ECG,max_heart_rate_achieved,excercise_induced_angina,ST_depression,peak_exercise_ST_segment,num_of_major_vessels,thal,heart_disease_diagnosis
0,54,1,4,125,216,0,0,140,0,0,NaN,NaN,NaN,1
1,55,1,4,158,217,0,0,110,1,2.5,2,NaN,NaN,1
2,54,0,3,135,304,1,0,170,0,0,1,0,3,0
3,48,0,3,120,195,0,0,125,0,0,NaN,NaN,NaN,0
4,50,1,4,120,0,0,1,156,1,0,1,NaN,6,3


In [6]:
test_df.head()

,age,sex,chest_pain_type,resting_blood_pressure,serum_cholestrol,fasting_blood_sugar,resting_ECG,max_heart_rate_achieved,excercise_induced_angina,ST_depression,peak_exercise_ST_segment,num_of_major_vessels,thal
0,60,1,2,160,267,1,1,157,0,0.5,2,NaN,NaN
1,61,1,4,148,203,0,0,161,0,0,1,1,7
2,54,1,4,130,242,0,0,91,1,1,2,NaN,NaN
3,48,1,4,120,260,0,0,115,0,2,2,NaN,NaN
4,57,0,1,130,308,0,0,98,0,1,2,NaN,NaN


## Data Preprocessing

### Impute Null Data

to make it simple, we encode null values with the median (numerical features) and mode (mode features)

In [7]:
train_df['resting_blood_pressure'].fillna(train_df['resting_blood_pressure'].dropna().median(),inplace=True)
train_df['serum_cholestrol'].fillna(train_df['serum_cholestrol'].dropna().median(),inplace=True)
train_df['fasting_blood_sugar'].fillna(train_df['fasting_blood_sugar'].dropna().mode(),inplace=True)
train_df['resting_ECG'].fillna(train_df['resting_ECG'].dropna().mode(),inplace=True)
train_df['max_heart_rate_achieved'].fillna(train_df['max_heart_rate_achieved'].dropna().median(),inplace=True)
train_df['excercise_induced_angina'].fillna(train_df['excercise_induced_angina'].dropna().mode(),inplace=True)
train_df['ST_depression'].fillna(train_df['ST_depression'].dropna().median(),inplace=True)
train_df['peak_exercise_ST_segment'].fillna(train_df['peak_exercise_ST_segment'].dropna().median(),inplace=True)
train_df['num_of_major_vessels'].fillna(train_df['num_of_major_vessels'].dropna().mode(),inplace=True)
train_df['thal'].fillna(train_df['thal'].dropna().mode(),inplace=True)

train_df.head()

,age,sex,chest_pain_type,resting_blood_pressure,serum_cholestrol,fasting_blood_sugar,resting_ECG,max_heart_rate_achieved,excercise_induced_angina,ST_depression,peak_exercise_ST_segment,num_of_major_vessels,thal,heart_disease_diagnosis
0,54,1,4,125,216,0,0,140,0,0,2,0,3,1
1,55,1,4,158,217,0,0,110,1,2.5,2,NaN,NaN,1
2,54,0,3,135,304,1,0,170,0,0,1,0,3,0
3,48,0,3,120,195,0,0,125,0,0,2,NaN,NaN,0
4,50,1,4,120,0,0,1,156,1,0,1,NaN,6,3


### Encode Categorical Data

General rules:
- Ordinal categorical type -> label encoding
- Nominal categorical type -> one-hot encoding

***TODO***

## Save Processed Data

Save the processed data to the ../raw/processed folder

In [8]:
PROCESSED_DATA_PATH = '../data/processed'
train_df.to_csv('{}/processed_data.csv'.format(PROCESSED_DATA_PATH),index=False)